In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from lib.pred_utils import *
import pandas as pd
from tqdm import tqdm

In [3]:
## define paths for models Using MaskRCNN for detection and Resnet34 for point net 
#maskrcnn="lib/model_weights/unzip_weights/model_final.pth"
#resnet34="lib/model_weights/unzip_weights/sc_v1_v2.pkl"
maskrcnn="lib/model_weights/31-10-21-weights/model_final.pth"
resnet34="lib/model_weights/31-10-21-weights/points.pkl"
img_path="filter_felda_s3_images"
save_path="results"

In [4]:
#Initialize MASKRCNN
#Hard coded the model config file in predutils change to use model zoo currrently not workign with custom detectron2
pepp=detectroninference(maskrcnn)

The checkpoint state_dict contains keys that are not used by the model:
  pixel_mean
  pixel_std


In [5]:
# Initialize Resnet34
learn=fast_AILearner(resnet34)

In [6]:
all_imgs=[]
fnames=[]
for file_index,filename in enumerate(os.listdir(img_path)):
    if filename.endswith(".tar") or filename.endswith(".json") or filename.endswith(".ipynb_checkpoints"):
        continue 
    f_p=os.path.join(img_path,filename)
    fnames.append(filename)
    all_imgs.append(f_p)

In [7]:
print(f"Number of Images in the directory:{len(all_imgs)}")

Number of Images in the directory:5078


In [8]:
# Set ppx and ppy which are 
# ppx=(length in cm/mm across x)/(number of pixels across x)
# ppy=(length in cm/mm across y)/(number of pixels across y)
# If they are unknown keep them None the results will be in pixels otherwise dimensions will have same units as measurement
#ppx=169/900
#ppy=126/669
#ppx,ppy

In [9]:
ppx=31.115/956
ppy=23.177/712
ppx,ppy

(0.03254707112970711, 0.03255196629213483)

In [10]:
def ray_trace_segment(img, init, direction="y", to_origin=1):
    """Ray trace from the inital points to some end  like end of non zero pixels in image. Is useful for fruits that turns and make an S shape
    Inputs and outputs are different based on x or y. May be better to write a differnt function for both"""
    assert len(img.shape) == 2, "Should be Gray scale image"
    h, w = img.shape
    if direction == "y":
        # Go downwards
        increment = 1
        x = init
        y = 0
        segments = []
        # Go along the height of image to find segments
        while y < h:
            firstPoint = None
            secondPoint = None
            if img[y, x]:  # find first non zero point
                firstPoint = (x, y)
                while y < h and img[y, x]:
                    y += increment
                # find last non zero point for that x, Replace below 4 with some more dynamic threshold
                if y - firstPoint[1] > 4:
                    secondPoint = (x, y)
                if firstPoint and secondPoint:
                    segments.append(
                        [firstPoint, secondPoint, secondPoint[1] - firstPoint[1]]
                    )
            y += increment
    elif direction == "x":
        x, y = init
        if to_origin:
            # Go Left
            increment = -1
        else:
            # Go Right
            increment = 1
        segments = []
        x += increment
        while x < w and x >= 0:
            if img[y, x]:
                segments.append((x, y))
            x += increment
    return segments

In [11]:
def min_max_segment(ref, xs, ys):
    """Min Max of fruit"""
    segments = []
    m_p = np.argwhere(xs == ref)
    y_p = ys[m_p]
    ymax = np.max(y_p)
    ymin = np.min(y_p)
    segments.append([(ref, ymin), (ref, ymax), ymax - ymin])
    return segments

In [12]:
def find_max_height(xs, ys):
    """Max height of fruit"""
    left_index = np.argwhere(xs == xs[0])
    y_lefts = set(ys[left_index].flatten())
    right_index = np.argwhere(xs == xs[-1])
    y_rights = set(ys[right_index].flatten())
    y_lefts = list(y_lefts)
    # print(f"The length of lengths is {len(y_lefts)}")
    y = y_lefts[len(y_lefts) // 2]
    height_points = [(xs[0], y), (xs[-1], y)]
    height = xs[-1] - xs[0]
    return height_points, height

In [13]:
def find_max_height_revised(xs, ys):
    """Max height of fruit"""
    # Loop throgh all ys and find min and max x
    assert len(xs)==len(ys), "Lengths of xs and ys are not equal"
    unique_y=sorted(set(ys))
    height_points=[]
    maxheight=-1
    height_points=None
    for y in unique_y:
        selected_index=np.argwhere(ys==y)
        selected_xs=xs[selected_index]
        xmin=np.min(selected_xs)
        xmax=np.max(selected_xs)
        height=xmax-xmin
        if height>maxheight:
            height_points=[(xmin, y), (xmax, y)]      
            maxheight=height
    #print(f"k"*100)
    #print(height_points)
    #print(maxheight)
    return height_points, maxheight

In [14]:
def find_mid_height_width(tops, bottoms):
    """Mid height width of fruit"""
    mid_height = len(tops) // 2
    width_points = [tops[mid_height], bottoms[mid_height]]
    width = bottoms[mid_height][1] - tops[mid_height][1]
    return width_points, width

In [15]:
def find_mid_width_height(xs, ys):
    """Mid width height of fruit"""
    y_sorted = sorted(set(ys))
    mid_y = y_sorted[len(y_sorted) // 2]
    y_indexs = np.argwhere(ys == mid_y)
    matched_xs = xs[y_indexs]
    min_x = int(min(matched_xs))
    max_x = int(max(matched_xs))
    height_points = [(min_x, mid_y), (max_x, mid_y)]
    height = max_x - min_x
    # print(f"Height Points {height_points}")
    return height_points, height

In [16]:
def find_max_width(tops, bottoms):
    """Max width of fruit"""
    max_index, max_width = None, -1
    for i in range(len(tops)):
        width = bottoms[i][1] - tops[i][1]
        if width > max_width:
            max_width = width
            max_index = i
    width_points = [tops[max_index], bottoms[max_index]]
    return width_points, max_width


In [17]:
def find_curve_length(top, bottom, mask,ign_pct=15):
    """Curved backbone length of fruit"""
    c_length = []
    # curved length causes issues at the borders so ignoring length at borders
    ign_pct = ign_pct / 100
    for i in range(int(len(top) * ign_pct), int(len(top) * (1 - ign_pct))):
        c_length.append((top[i][0], round((top[i][1] + bottom[i][1]) / 2)))
    # Ray trace to extend the curve length on left
    segment_left = ray_trace_segment(mask, c_length[0], direction="x", to_origin=1)
    c_length = segment_left + c_length
    # Ray trace to extend the curve length on right
    segment_right = ray_trace_segment(mask, c_length[-1], direction="x", to_origin=0)
    c_length = c_length + segment_right
    return c_length

In [18]:
def ellipse_fitting(mask):
    """Ellipse fitting of fruit"""
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    contours = max(contours, key = cv2.contourArea)
    #assert len(contours) == 1, "More than one contours in ellipse mask"
    #cont = contours[0]
    cont=contours
    (x, y), (MA, ma), angle = cv2.fitEllipse(cont)
    x, y = int(x), int(y)
    MA, ma = int(MA), int(ma)
    ellipse_img = np.zeros_like(mask)
    #draw complete ellipse form angle 0 to 360
    cv2.ellipse(ellipse_img, (x, y), (MA // 2, ma // 2), angle, 0, 360, 255)
    elipse_contours, _ = cv2.findContours(ellipse_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    elipse_contour = max(elipse_contours, key = cv2.contourArea)
    #assert len(elipse_contours) == 1, "More than one contours in ellipse"
    #elipse_contour = elipse_contours[0]
    error = cv2.matchShapes(cont, elipse_contour, 1, 0.0)
    return (x,y,MA,ma,angle), error

In [19]:
def blockiness(top, bottom,ign_pct=20):
    """Blockiness at different points of fruits"""
    ign_pct = ign_pct / 100
    bottomindex = int(len(top) * ign_pct)
    midindex = int(len(top) // 2)
    topindex = int(len(top) * (1 - ign_pct))
    bottom_block = [top[bottomindex], bottom[bottomindex]]
    mid_block = [top[midindex], bottom[midindex]]
    top_block = [top[topindex], bottom[topindex]]
    return bottom_block, mid_block, top_block

In [20]:
def box_fit(mask):
    """Find bounding box of the detected cropped mask"""
    x, y, w, h = cv2.boundingRect(mask)
    return (x,y,w,h),w/h

In [21]:
from colormath.color_objects import LabColor, XYZColor, sRGBColor
from colormath.color_conversions import convert_color
def get_color(patch,points):
    b=np.mean(patch[[*points,0]])
    g=np.mean(patch[[*points,1]])
    r=np.mean(patch[[*points,2]])
    rgb=sRGBColor(r,g,b,is_upscaled=True)
    lab=convert_color(rgb, LabColor)
    return [rgb.get_upscaled_value_tuple(),lab.get_value_tuple()]
    #print(f"Average value of blue green and red are {b}, {g}, {r}")
    #print(f"RGB upscaled {rgb.get_upscaled_value_tuple()}")
    #print(f"LAB upscaled {lab.get_value_tuple()}")

In [22]:
def vis_phenotypes(
    perimeter,
    area,
    max_height,
    mid_width_height,
    max_width,
    mid_height_width,
    curved_length,
    bottom_block,
    mid_block,
    top_block,
    box,
    ellipse,
    img,
    line_width=5,
):
    """Visualization of phenotypes"""
    fig = plt.figure(figsize=(10, 20))
    nrows = 7
    ncols = 2
    vis_imgs = []

    vis_img = img.copy()
    for i in range(len(perimeter)):
        cv2.circle(vis_img, tuple(perimeter[i]), line_width, (255, 0, 0))
    vis_imgs.append(vis_img)

    vis_img = img.copy()
    cv2.line(vis_img, max_height[0], max_height[1], (255, 0, 0), line_width)
    vis_imgs.append(vis_img)

    vis_img = img.copy()
    cv2.line(vis_img, max_width[0], max_width[1], (0, 255, 0), line_width)
    vis_imgs.append(vis_img)

    vis_img = img.copy()
    cv2.line(vis_img, mid_width_height[0], mid_width_height[1], (0, 0, 255), line_width)
    vis_imgs.append(vis_img)

    vis_img = img.copy()
    cv2.line(
        vis_img, mid_height_width[0], mid_height_width[1], (255, 255, 0), line_width
    )
    vis_imgs.append(vis_img)

    vis_img = img.copy()
    for i in range(len(curved_length)):
        cv2.circle(vis_img, curved_length[i], line_width, (255, 0, 0))
    vis_imgs.append(vis_img)
    # Shape
    # max height to maximum width
    vis_img = img.copy()
    cv2.line(vis_img, max_height[0], max_height[1], (255, 0, 0), line_width)
    cv2.line(vis_img, max_width[0], max_width[1], (0, 255, 0), line_width)
    vis_imgs.append(vis_img)
    
    # Height mid width/ width mid height
    vis_img = img.copy()
    cv2.line(vis_img, mid_width_height[0], mid_width_height[1], (0, 0, 255), line_width)
    cv2.line(
        vis_img, mid_height_width[0], mid_height_width[1], (255, 255, 0), line_width
    )
    vis_imgs.append(vis_img)

    # Top blockiness/ Mid height Width
    vis_img = img.copy()
    cv2.line(vis_img, top_block[0], top_block[1], (0, 0, 255), line_width)
    cv2.line(
        vis_img, mid_height_width[0], mid_height_width[1], (255, 255, 0), line_width
    )
    vis_imgs.append(vis_img)

    # Bottom blockiness/ Mid height Width
    vis_img = img.copy()
    cv2.line(vis_img, bottom_block[0], bottom_block[1], (0, 0, 255), line_width)
    cv2.line(
        vis_img, mid_height_width[0], mid_height_width[1], (255, 255, 0), line_width
    )
    vis_imgs.append(vis_img)

    # Upper blockiness/ Bottom blockiness
    vis_img = img.copy()
    cv2.line(vis_img, top_block[0], top_block[1], (0, 0, 255), line_width)
    cv2.line(vis_img, bottom_block[0], bottom_block[1], (255, 255, 0), line_width)
    vis_imgs.append(vis_img)
    
   
    # Ellipse vis
    vis_img = img.copy()
    x,y,MA,ma,angle=ellipse
    cv2.ellipse(vis_img, (x, y), (MA // 2, ma // 2), angle, 0, 360, (255,0,0),line_width)
    vis_imgs.append(vis_img)
    
    # Box vis
    vis_img = img.copy()
    x,y,w,h=box
    vis_img = cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,0),line_width)
    vis_imgs.append(vis_img)

    labels = [
        "Perimeter",
        "Max height",
        "Max Width",
        "Mid Width Height",
        "Mid Height Width",
        "Curved Length",
        "max height/max width",
        "height mid-width/width mid-height",
        "Proximal Fruit Blokiness",
        "Distal Fruit Blokiness",
        "Fruit Shape Triangle",
        "Elipse",
        "Rectangle",
    ]
    
    assert len(labels)==len(vis_imgs), f"Label length {len(labels)}  and vis images {len(vis_imgs)} not equal"
    for i, patch in enumerate(vis_imgs):
        ax = fig.add_subplot(nrows, ncols, i + 1)
        ax.set_title(labels[i])
        ax.imshow(patch[..., ::-1])
    #fig.subplots_adjust(hspace=0.5, wspace=0.04)
    fig.tight_layout()
    return fig,vis_imgs,labels


In [23]:
def eucledian_distance(points):
    length=0
    if ppx and ppy:
        for i in range(1,len(points)):
            length+=np.sqrt((((points[i][0]-points[i-1][0])*ppx)**2) +  (((points[i][1]-points[i-1][1])*ppy)**2))
    else:
        for i in range(1,len(points)):
            length+=np.sqrt(((points[i][0]-points[i-1][0])**2) +  ((points[i][1]-points[i-1][1])**2))
    return length

In [24]:
def convert_pixels_to_measure(area,perimeter,mid_height_width_p,max_width_p,mid_width_height_p,max_height_p,c_length):
    """Convert pixels to real units cmm/mm if ppx and ppy are defined otherwise find lengths of curves"""
    if ppx and ppy:
        area=area*ppx*ppy
        perimeter=perimeter*ppx
    mid_height_width=eucledian_distance(mid_height_width_p)
    max_width=eucledian_distance(max_width_p)
    mid_width_height=eucledian_distance(mid_width_height_p)
    max_height=eucledian_distance(max_height_p)
    #print(f"Max height eucledian {max_height}")
    c_length=eucledian_distance(c_length)
    return (area,perimeter,mid_height_width,max_width,mid_width_height,max_height,c_length)

In [25]:
def phenotype_measurement_pepper(img, patch,skip_outliers=None, pct_width=0.4,vis=False):
    contours, _ = cv2.findContours(img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    # Get the maximum contour if multiple contours exist per fruit
    contours = max(contours, key = cv2.contourArea)
    points=contours.squeeze()
    # points are perimeter
    points = np.array(sorted(points, key=lambda x: x[0]))
    # Perimeter fruit
    perimeter = cv2.arcLength(contours,True)
    # Area fruit
    area = cv2.contourArea(contours)
    area_points = np.where(img != 0)
    #color first tuple is rgb, second is LAB
    fruit_colors=get_color(patch.copy(),area_points)
    #contour points are reverse of image points x is in 0 and y is in 1 dimension
    xs = points[:, 0]
    ys = points[:, 1]
    # sort xs
    u_xs = sorted(set(xs))
    top = []
    bottom = []
    widths = []
    # lengths
    max_height_p, max_height_v = find_max_height_revised(xs, ys)
    #print(f"Max height value {max_height_p}")
    #print(f"Max height value {max_height_v}")
    #print(f"PPx and PPy {ppx} {ppy}")
    mid_width_height_p, mid_width_height_v = find_mid_width_height(xs, ys)

    for i, x in enumerate(u_xs):
        # segments=ray_trace_segment(img,x)
        segments = min_max_segment(x, xs, ys)
        for segment in segments:
            top.append(segment[0])
            bottom.append(segment[1])
            widths.append(segment[2])

    assert (
        len(top) == len(bottom) == len(widths)
    ), "The top, bottom and width points are not equal"

    # widths and curve lengths
    max_width_p, max_midth_v = find_max_width(top, bottom)
    mid_height_width_p, mid_height_width_v = find_mid_height_width(top, bottom)
    c_length = find_curve_length(top, bottom, img)
    b_block, m_block, t_block = blockiness(top, bottom)
    
    # metric 14
    ellipse,ellipse_err=ellipse_fitting(img)
    #meric 15
    box,box_aspect=box_fit(img)
    if vis:
        fig,vis_images,labels = vis_phenotypes(
            points,
            area_points,
            max_height_p,
            mid_width_height_p,
            max_width_p,
            mid_height_width_p,
            c_length,
            b_block,
            m_block,
            t_block,
            box,
            ellipse,
            patch.copy(),
        )
    else:
        fig,vis_images,labels=None,None,None
    # median_width=statistics.median(widths)
    area,perimeter,mid_height_width,max_width,mid_width_height,max_height,c_length=\
    convert_pixels_to_measure(area,perimeter,mid_height_width_p,max_width_p,mid_width_height_p,max_height_p,c_length)
    # shape metrics
    #metric 8
    maxheight_to_maxwidth=max_height/max_width
    #metrix 9
    mid_width_height_to_mid_height_width=mid_width_height/mid_height_width
    # metric 10
    c_length_to_mid_height_width=c_length/mid_height_width
    # metric 11
    proximal_blockiness=eucledian_distance(t_block)/mid_height_width
    # metric 12
    distal_blockiness=eucledian_distance(b_block)/mid_height_width
    # metric 13
    fruit_triangle=eucledian_distance(t_block)/eucledian_distance(b_block)
    
    return (
        area,
        perimeter,
        mid_height_width,
        max_width,
        mid_width_height,
        max_height,
        c_length,
        maxheight_to_maxwidth,
        mid_width_height_to_mid_height_width,
        c_length_to_mid_height_width,
        proximal_blockiness,distal_blockiness,fruit_triangle,
        ellipse_err,box_aspect,
        fruit_colors,
        fig,vis_images,labels,
        # Added for validation and debugging
        max_height_p,
        mid_height_width_p
        
    )

In [26]:
def draw_text(img,length,width):
    # font
    font = cv2.FONT_HERSHEY_SIMPLEX
    # org
    org1 = (int(img.shape[1]*0.3), int(img.shape[0]*0.4))
    org2 = (int(img.shape[1]*0.3), int(img.shape[0]*0.7))
    # fontScale
    fontScale = 1
    color = (255, 0, 0)
    thickness = 2
    img = cv2.putText(img, f"l: {length}", org1, font, fontScale, color, thickness, cv2.LINE_AA)
    img = cv2.putText(img, f"w: {width}", org2, font, fontScale, color, thickness, cv2.LINE_AA)
    return img

In [27]:
def run_inference(img):
    if img is not None:
        detected_cucumber,all_masks,all_patches,boxes,*_=pepp.pred(img)
    else:
        print(f"Invalid Image skipping!!")
        return
    cropedPatches=[]
    cropedmasks=[]
    croppedboxes = []
    for i,patch in enumerate(all_patches):
        #print(patch.shape)
        x, y, w, h = cv2.boundingRect(cv2.cvtColor(patch, cv2.COLOR_BGR2GRAY))
        newImg = patch[y:y+h, x:x+w]
        mask=all_masks[i][y:y+h, x:x+w]
        mask = np.where(mask < 200, 0, 255).astype(np.uint8)
        cropedPatches.append(newImg)
        cropedmasks.append(mask)
        croppedboxes.append((x, y, w, h))
        #break
        
    predPoints={}
    for i,crop in enumerate(cropedPatches):
        img = PILImage.create(crop[...,::-1])
        pred=learn.predict(img)
        preds=scale_predictions(pred,img.shape)
        predPoints[i]=[tuple(preds[0].round().long().numpy()),tuple(preds[1].round().long().numpy())]
    
    # Correction
    corrected_imgs=apply_rotation(cropedPatches,predPoints)
    corrected_masks=apply_rotation(cropedmasks,predPoints)
        
    return (
        detected_cucumber,
        corrected_imgs,
        corrected_masks,
        croppedboxes,
        cropedPatches,
        predPoints,
        #original boxes are returned to get Gt based on location of boxes
        boxes
    )

In [28]:
def save_results_figs(
    detected_cucumber, cropedPatches, predPoints, corrected_imgs, fname, save_path
):
    """Save figures of results to visualize
    Args:
        detected_cucumber ([type]): Detection result from detectron2
        cropedPatches ([type]): Croped patch of original Image
        predPoints ([type]): predicted keypoints adjusted according to cropped patches
        corrected_imgs ([type]): orientation corrected Image
        fname ([type]): Filename for saving postfix
        save_path ([type]): "Path to save"
    """
    os.makedirs(save_path, exist_ok=True)
    fig = plt.figure(figsize=(10, 10))
    plt.title("Detected Pepper MASKRCNN", loc="center")
    plt.imshow(detected_cucumber[..., ::-1])
    fig.savefig(os.path.join(save_path, f"segmented_{fname}"))
    # show croped images
    fig = plt.figure(figsize=(10, 10))
    plt.title("Cropped", loc="center")
    nrows = np.ceil(len(cropedPatches) / 2)
    ncols = 2
    for i, patch in enumerate(cropedPatches):
        ax = fig.add_subplot(nrows, ncols, i + 1)
        ax.imshow(patch[..., ::-1])
    fig.savefig(os.path.join(save_path, f"Cropped_{fname}"))
    fig = check_points(cropedPatches, predPoints)
    fig.savefig(os.path.join(save_path, f"Points_{fname}"))
    fig = plt.figure(figsize=(10, 10))
    # plt.title("Corrected Orientation",loc='center')
    # plt.rcParams['figure.figsize'] = [45, 45]
    nrows = np.ceil(len(corrected_imgs) / 2)
    ncols = 2
    for i, correct_img in enumerate(corrected_imgs):
        ax = fig.add_subplot(nrows, ncols, i + 1)
        ax.imshow(correct_img[..., ::-1])
    fig.savefig(os.path.join(save_path, f"corrected_{fname}"))
    ## Concatenate Results
    merge_scans = os.path.join(save_path + "merge_results")
    os.makedirs(merge_scans, exist_ok=True)
    img_det = cv2.imread(os.path.join(save_path, f"segmented_{fname}"))
    img_crop = cv2.imread(os.path.join(save_path, f"Cropped_{fname}"))
    img_points = cv2.imread(os.path.join(save_path, f"Points_{fname}"))
    img_corrected = cv2.imread(os.path.join(save_path, f"corrected_{fname}"))
    joined_img = np.concatenate([img_det, img_crop, img_points, img_corrected], axis=1)
    cv2.imwrite(os.path.join(merge_scans, f"{fname}"), joined_img)

In [29]:
def save_results_cv(
    detected_cucumber, cropedPatches, predPoints, corrected_imgs, fname, save_path
):
    """Save images of results to visualize
    Args:
        detected_cucumber ([type]): Detection result from detectron2
        cropedPatches ([type]): Croped patch of original Image
        predPoints ([type]): predicted keypoints adjusted according to cropped patches
        corrected_imgs ([type]): orientation corrected Image
        fname ([type]): Filename for saving postfix
        save_path ([type]): "Path to save"
    """
    print(f"Save path {save_path}")
    os.makedirs(save_path, exist_ok=True)
    
    cv2.imwrite(os.path.join(save_path, f"Segmented_{fname}"),detected_cucumber)
    # show croped images
    for i, patch in enumerate(cropedPatches):
        cv2.imwrite(os.path.join(save_path, f"Cropped_{i}_{fname}"),patch)
    
    point_imgs=check_points_imgs(cropedPatches, predPoints)
    
    for i, point_img in enumerate(point_imgs):
        cv2.imwrite(os.path.join(save_path, f"Points_{i}_{fname}"),point_img)
    
    for i, correct_img in enumerate(corrected_imgs):
        cv2.imwrite(os.path.join(save_path, f"Corrected_{i}_{fname}"),correct_img)

In [30]:
def gtfinder(imgname,boxes):
    """Assign Gt in the same order as the bboxes, append also the fruit numbers for error analysis"""
    all_points=valid_coordinates[imgname]
    #print(all_points)
    # gts is a list of tuples contaitning length and widths of fruits to be verified
    gts={}
    #print(f"length of pred boxes {len(boxes)}")
    for k,v in all_points.items():
        for i,box in enumerate(boxes):
            if (v[0]>=(box[0]) and  v[0]<=(box[2]) and  v[1]>=(box[1]) and  v[1]<=(box[3])):
                #print(f"Found Match!!")
                #print(f"Box is {box}")
                #print(f"match is {v}")
                gt_length=val_df.loc[val_df["id"] == k]["max_len"].values[0]
                if math.isnan(gt_length):
                    gt_length=None
                gt_width=val_df.loc[val_df["id"] == k]["mid_width"].values[0]
                if math.isnan(gt_width):
                    gt_width=None
                gts[i]=[k,gt_length,gt_width]
                break
    #print(f"length of gt points {len(all_points)}")
    #print(f"length of gt founder points {len(gts)}")
    return gts

In [31]:
skipped_nan=0
def error(preds,gts,pred_ls_ps,pred_ws_ps):
    global skipped_nan
    fruits_idx=[]
    length_errs=[]
    width_errs=[]
    org_l=[]
    org_w=[]
    pred_l=[]
    pred_w=[]
    pred_l_p=[]
    pred_w_p=[]
    #print(gts)
    #for i in range(len(gts)):
    for i in gts.keys():
        if gts[i][1] is None or gts[i][2] is None:
            skipped_nan+=1
            continue
        #ignore ambigious predictions
        #if gts[i][0] in [43,84,67]:
        #    continue
        l_err=abs(preds[i][0]-gts[i][1])
        w_err=abs(preds[i][1]-gts[i][2])
        length_errs.append(l_err)
        width_errs.append(w_err)
        fruits_idx.append(gts[i][0])
        org_l.append(gts[i][1])
        org_w.append(gts[i][2])
        pred_l.append(preds[i][0])
        pred_w.append(preds[i][1])
        pred_l_p.append({gts[i][0]:pred_ls_ps[i]})
        pred_w_p.append({gts[i][0]:pred_ws_ps[i]})
    return length_errs,width_errs,fruits_idx,org_l,org_w,pred_l,pred_w,pred_l_p,pred_w_p

In [32]:
import time

In [ ]:
startsec=time.time()
results={}
globalIndex=0
empty_images=0
length_errors=[]
width_errors=[]
fruit_idxs=[]
org_ls=[]
org_ws=[]
pred_ls=[]
pred_ls_ps=[]
pred_ws=[]
pred_ws_ps=[]
plot_names=[]
vis_results=False

for index in tqdm(range(len(all_imgs)),total=len(all_imgs)):    
    img=cv2.imread(all_imgs[index])
    (detected,
    corrected_imgs,
    corrected_masks,
    cropped_boxes,
    cropedPatches,
    predPoints,boxes) = run_inference(img)
    #print(f"Boxes are {boxes}")
    print(f"Processing file {fnames[index]}")
    #Append plot names to count plot
    plot_name=fnames[index].split("_")[0]
    if plot_name not in plot_names:
        plot_names.append(plot_name)
    # Get length and width of GT for comparison
    #gts=gtfinder(fnames[index],boxes)
    #print(f"Found Ground truth of {gts}")
    if not len(corrected_masks):
        empty_images+=1
    preds=[]
    pred_ls_ps_inter=[]
    pred_ws_ps_inter=[]
    for j, mask in enumerate(corrected_masks):
        globalIndex+=1
        assert len(mask.shape) == 2, "Should be Gray scale Mask"
        (
        area,
        perimeter,
        mid_height_width,
        max_width,
        mid_width_height,
        max_height,
        c_length,
        maxheight_to_maxwidth,
        mid_width_height_to_mid_height_width,
        c_length_to_mid_height_width,
        proximal_blockiness,distal_blockiness,fruit_triangle,
        ellipse_err,box_aspect,
        fruit_colors,
        fig,phen_imgs,labels,tmp_pred_ls_ps,tmp_pred_ws_ps
        ) = phenotype_measurement_pepper(mask, corrected_imgs[j],skip_outliers=True,vis=vis_results)
        
        ## Debug color
        #print(f"Fruit colors rgb is {fruit_colors[0]}, LAB is {fruit_colors[1]}")
            
        # for validation and debugging
        pred_ls_ps_inter.append(tmp_pred_ls_ps)
        pred_ws_ps_inter.append(tmp_pred_ws_ps)
        #Save predicion for error calcualtions
        
        preds.append([max_height,mid_height_width])
        
        #print(f"Saving phenotypes fig!!")
        if vis_results:
            fig.savefig(os.path.join(save_path, f"Phenotype_{j}_{fnames[index]}"))
        #save results
        if "Image" not in results: 
            results["Image"]=[fnames[index]]
            results["Area"]=[area]
            results["Perimeter"]=[perimeter]
            results["Mid_Width"]=[mid_height_width]
            results["Max_Width"]=[max_width]
            results["Mid_Height"]=[mid_width_height]
            results["Max_Height"]=[max_height]
            results["Curved_Height"]=[c_length]
            results["Maxheight_to_maxwidth"]=[maxheight_to_maxwidth]
            results["Midheight_to_midwidth"]=[mid_width_height_to_mid_height_width]
            results["Curveheight_to_midwidth"]=[c_length_to_mid_height_width]
            results["Proximal_blockiness"]=[proximal_blockiness]
            results["Distall_blockiness"]=[distal_blockiness]
            results["Fruit_triangle"]=[fruit_triangle]
            results["Ellipse_Error"]=[ellipse_err]
            results["Box_Aspect"]=[box_aspect]
        else:
            results["Image"].append(fnames[index])
            results["Area"].append(area)
            results["Perimeter"].append(perimeter)
            results["Mid_Width"].append(mid_height_width)
            results["Max_Width"].append(max_width)
            results["Mid_Height"].append(mid_width_height)
            results["Max_Height"].append(max_height)
            results["Curved_Height"].append(c_length)
            results["Maxheight_to_maxwidth"].append(maxheight_to_maxwidth)
            results["Midheight_to_midwidth"].append(mid_width_height_to_mid_height_width)
            results["Curveheight_to_midwidth"].append(c_length_to_mid_height_width)
            results["Proximal_blockiness"].append(proximal_blockiness)
            results["Distall_blockiness"].append(distal_blockiness)
            results["Fruit_triangle"].append(fruit_triangle)
            results["Ellipse_Error"].append(ellipse_err)
            results["Box_Aspect"].append(box_aspect)
    # Ideally the detected objects should have all the 
    #assert len(valid_coordinates[fnames[index]])==len(gts),f"Length of Expected GT {fnames[index]}, Length of Matched Gt is {len(gts)}"
    
    #if fnames[index]=="SCALE_CAM_VALIDATION_100_PEPPERS_211022_150446_116904_001.jpg":
    #    break
    #length_err,width_err,fruit_idx,org_l,org_w,pred_l,pred_w,pred_ls_ps_inter,pred_ws_ps_inter=error(preds,gts,pred_ls_ps_inter,pred_ws_ps_inter)
    
    # save figures results
    if vis_results:
        save_results_cv(
        detected,
        cropedPatches,
        predPoints,
        corrected_imgs,
        fnames[index],
        save_path+"/inter",)
    #if globalIndex>1000:
    #    break
endsec=time.time()

  0%|          | 0/5078 [00:00<?, ?it/s]/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/torch/functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
  0%|          | 1/5078 [00:01<1:45:09,  1.24s/it]

Processing file P00000000879559568172638_211208_210540_838897.jpg


  0%|          | 2/5078 [00:01<1:10:29,  1.20it/s]

Processing file P00000000879559568172712_211208_194348_821563.jpg


  0%|          | 3/5078 [00:02<59:23,  1.42it/s]  

...

Processing file P00000000879559568172247_211208_125610_067290.jpg


 44%|████▍     | 2236/5078 [38:03<45:22,  1.04it/s]

Processing file P00000000879559568172287_211207_201211_579775.jpg


 44%|████▍     | 2237/5078 [38:04<47:51,  1.01s/it]

Processing file P00000000879559568171890_211207_133726_293844.jpg


In [ ]:
#plot_names

In [ ]:
print(f"Total elapsed time in minutes {(endsec-startsec)*0.0166667}")
print(f"Average elapsed time per image in secs {(endsec-startsec)/len(all_imgs)}")

In [ ]:
print(f"Total Plots Processed {len(plot_names)}")
print(f"Total Fruits Processed {globalIndex}")
print(f"Total Empty images {empty_images}")
print(f"Average fruits per image {globalIndex/(len(all_imgs)-empty_images)}")

### Save Results

In [ ]:
results_df=pd.DataFrame.from_dict(results,orient='index').transpose()
results_df

In [ ]:
results_df.describe()

In [ ]:
results_df.to_excel('results_felda.xlsx')

## Merge Results

In [ ]:
results.keys()

In [ ]:
ex_p_names=[i.split("_")[0] for i in results['Image']]
p_names=set(ex_p_names)
o_p_names=set(plot_names)
len(p_names),len(o_p_names)
## These plots contain no useful information
o_p_names-p_names

In [ ]:
average_fruits=[]
agg_results={}
name_map={'Area':"FRAREA", 'Perimeter':"FRPER", 'Mid_Width':"FRWMH", 'Max_Width':"FRMAXW", 'Mid_Height':"FRMHMW", 'Max_Height':"FRMH", 'Curved_Height':"FRSPH", 'Maxheight_to_maxwidth':"FRSIEMAX", 'Midheight_to_midwidth':"FRSIEMID", 'Curveheight_to_midwidth':"FRCSI", 'Proximal_blockiness':"FRPB", 'Distall_blockiness':"FRDB", 'Fruit_triangle':"FRSTRI", 'Ellipse_Error':"FRELIP", 'Box_Aspect':"FRRECT"}
#name_map.keys()

In [ ]:
for name in p_names:
    n_fruits=0
    all_phen=[[] for i in range(len(name_map.keys()))]
    for i,n in enumerate(results["Image"]):
        if name==n.split("_")[0]:
            n_fruits+=1
            for j,pn in enumerate(name_map.keys()):
                all_phen[j].append(results[pn][i])
    print(f"Fruits in {name} are {n_fruits}")
    assert n_fruits==len(all_phen[0])==len(all_phen[-1])
    average_fruits.append(n_fruits)
    if "PLOT BID" not in agg_results: 
        agg_results["PLOT BID"]=[name]
        for l,(k,v) in enumerate(name_map.items()): 
            agg_results[v]=[np.mean(all_phen[l])]
            agg_results[v+"_CNT"]=[len(all_phen[l])]
            agg_results[v+"_MAX"]=[np.max(all_phen[l])]
            agg_results[v+"_MIN"]=[np.min(all_phen[l])]
            agg_results[v+"_SD"]=[np.std(all_phen[l])]
    else:
        agg_results["PLOT BID"].append(name)
        for l,(k,v) in enumerate(name_map.items()): 
            agg_results[v].append(np.mean(all_phen[l]))
            agg_results[v+"_CNT"].append(len(all_phen[l]))
            agg_results[v+"_MAX"].append(np.max(all_phen[l]))
            agg_results[v+"_MIN"].append(np.min(all_phen[l]))
            agg_results[v+"_SD"].append(np.std(all_phen[l]))

In [ ]:
agg_df=pd.DataFrame.from_dict(agg_results,orient='index').transpose()
agg_df

In [ ]:
agg_df.describe()

In [ ]:
agg_df.to_excel('aggregate_results_felda.xlsx')